# 导入函数库

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras  # 在TF2.0版本中,已经是自带Keras了,所以不需要额外安装
import tensorflow_datasets as tfds # 这个是之前说过的Tensorflow Datasets

# 如果出现显存不够的错误,把这个代码加上

In [2]:
# 加入下面这个代码片段主要是因为TF2.0对RTX20系列显卡支持不是很好,容易爆显存,所以设置成用多少占多少显存,而不是一次性全占了(默认)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 定义网络结构

## 一些参数设置

In [3]:
layers = tf.keras.layers
models = tf.keras.models

## 定义LeNet5用于手势识别

In [4]:
def LeNet5(input_shape=(32, 32, 3),
         classes=3):
    img_input = layers.Input(shape=input_shape)  # 输入节点
    
    x = layers.Conv2D(64, (5, 5),
                      activation='relu',
                      padding='valid',
                      name='conv1')(img_input)
    x = layers.BatchNormalization()(x)
    x = layers.AveragePooling2D()(x)
    x = layers.Conv2D(64, (5, 5),
                      activation='relu',
                      padding='valid',
                      name='conv2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.AveragePooling2D()(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu', name='fc1')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation='relu', name='fc2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)
    
    inputs = img_input  # inputs是输入节点
    outputs = x # outputs是输出节点
    model = models.Model(inputs, outputs , name='lenet5')  # 生成一个Model, 需要指定输入和输出
    
    return model

## 声明一个LeNet5模型实例

In [5]:
model = LeNet5()

## 记录模型训练过程中的日志

In [6]:
log_dir = './logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## 进行数据增强

In [7]:
def convert(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    image = tf.image.resize(image, size=[32, 32])
    return image, label

def augment(image,label):
    image,label = convert(image, label)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    return image,label

## 数据读取并预处理

In [8]:
# 此处使用tfds的方式构建data pipeline
(raw_test, raw_train), metadata = tfds.load(
    'rock_paper_scissors', # 数据集名称，这个是手势识别分类数据集，共3个类别
    split=['test', 'train'], # 这里的raw_test和split的'test'对应，raw_train和split的'train'对应
    with_info=True, # 这个参数和metadata对应
    as_supervised=True, # 这个参数的作用是返回tuple形式的(input, label),举个例子，raw_test=tuple(input, label)
    shuffle_files=True  # 对数据进行扰乱操作，可以自己体会下设置成False时，下面imshow的时候的结果差别
)

BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 2520

# 可以体验下这里是否加prefetch(tf.data.experimental.AUTOTUNE)和cache()的区别，对训练速度，以及CPU负载有影响
train_batches = raw_train.shuffle(SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True).map(augment).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = raw_test.map(convert).batch(BATCH_SIZE)

## 进行模型训练

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(
    train_batches,
    epochs=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
630/630 [==============================] - 2s 2ms/step - loss: 0.9219 - accuracy: 0.6250
Epoch 2/20
630/630 [==============================] - 2s 4ms/step - loss: 0.7872 - accuracy: 0.7702
Epoch 3/20
630/630 [==============================] - 2s 3ms/step - loss: 0.7379 - accuracy: 0.8135
Epoch 4/20
630/630 [==============================] - 3s 4ms/step - loss: 0.7118 - accuracy: 0.8440
Epoch 5/20
630/630 [==============================] - 2s 4ms/step - loss: 0.6924 - accuracy: 0.8667
Epoch 6/20
630/630 [==============================] - 2s 4ms/step - loss: 0.6729 - accuracy: 0.8837
Epoch 7/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6753 - accuracy: 0.8790
Epoch 8/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6524 - accuracy: 0.9071
Epoch 9/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6452 - accuracy: 0.9155
Epoch 10/20
630/630 [==============================] - 2s 4ms/step - loss: 0.6495 - accuracy: 0.9067

## Baseline的test acc

In [10]:
_, baseline_model_accuracy = model.evaluate(test_batches, verbose=1)
print('Baseline test accuracy: ', baseline_model_accuracy)

keras_file = './test.h5'
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to: ', keras_file)

93/93 [==============================] - 0s 2ms/step - loss: 0.6502 - accuracy: 0.9140
Baseline test accuracy:  0.9139785170555115
Saved baseline model to:  ./test.h5
